# Polar H10 ECG and ACC-datastream
10.2.2022, Sakari Lukkarinen<br>
Metropolia University of Applied Sciences<br>


- [Polar Device Data Stream ECG Python example by Pareek Nikhil](https://github.com/pareeknikhil/biofeedback/tree/master/Polar%20Device%20Data%20Stream/ECG)
- [Polar Device Data Stream ACC Python example by Pareek Nikhil](https://github.com/pareeknikhil/biofeedback/tree/master/Polar%20Device%20Data%20Stream/Accelerometer)
- [Creating a data stream with polar device - Blog post by Pareek Nikhil](https://towardsdatascience.com/creating-a-data-stream-with-polar-device-a5c93c9ccc59)
- [Bleak documentation](https://bleak.readthedocs.io/en/latest)
- [Polar SDK](https://www.polar.com/en/developers/sdk)
    - [Technical documentation](https://github.com/polarofficial/polar-ble-sdk/tree/master/technical_documentation)
    - [Polar H10 heart rate sensor](https://www.polar.com/us-en/products/accessories/h10_heart_rate_sensor)


## Instructions

1. Change these parameters first. 
2. Select: Run  > Run all cells.
3. Scroll down and see the live recording.
4. Observe how many seconds it takes the streaming to be live.
5. Wait until the first test is over.
6. Change the `WAIT_N_SECONDS', if necessary.
7. To make another recording, select: Kernel > Restart Kernel and Run All Cells.

In [ ]:
## This is the device MAC ID, please update with your device ID
ADDRESS = "E3:7B:AA:2D:D5:CD"

# Recording length in seconds
REC_LENGTH = 60

# Wait period before recording starts, typical value = 20..25
WAIT_N_SECONDS = 2

# Data is stored to these files
ECG_DATAFILE = "ecg_data.csv"
ACC_DATAFILE = "acc_data.csv"

## Setup

In [ ]:
## Remove comment and run, if any of these libraries are missing from your installation
# !pip install bleak
# !pip install nest_asyncio
# !pip install seaborn

In [ ]:
%matplotlib inline
import os
import sys
import time
import math
import asyncio
import nest_asyncio
import signal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from IPython.display import display, clear_output
import seaborn as sns
from bleak import BleakClient
from bleak.uuids import uuid16_dict

sns.set_theme()

nest_asyncio.apply()

In [ ]:
""" Predefined UUID (Universal Unique Identifier) mapping are based on Heart Rate GATT service Protocol that most
Fitness/Heart Rate device manufacturer follow (Polar H10 in this case) to obtain a specific response input from 
the device acting as an API """
uuid16_dict = {v: k for k, v in uuid16_dict.items()}

## UUID codes
MODEL_NBR_UUID = "00002a24-0000-1000-8000-00805f9b34fb"
MANUFACTURER_NAME_UUID = "00002a29-0000-1000-8000-00805f9b34fb"
BATTERY_LEVEL_UUID = "00002a19-0000-1000-8000-00805f9b34fb"
PMD_SERVICE = "FB005C80-02E7-F387-1CAD-8ACD2D8DF0C8"
PMD_CONTROL = "FB005C81-02E7-F387-1CAD-8ACD2D8DF0C8"
PMD_DATA = "FB005C82-02E7-F387-1CAD-8ACD2D8DF0C8"

## Polar specific codes
ECG_WRITE = bytearray([0x02, 0x00, 0x00, 0x01, 0x82, 0x00, 0x01, 0x01, 0x0E, 0x00])
ACC_WRITE = bytearray([0x02, 0x02, 0x00, 0x01, 0xC8, 0x00, 0x01, 0x01, 0x10, 0x00, 0x02, 0x01, 0x08, 0x00])
START_PPG_STREAM = bytearray([0x02, 0x01, 0x00, 0x01, 0xC8, 0x00, 0x01, 0x01, 0x10, 0x00, 0x02, 0x01, 0x08, 0x00])
START_ECG_STREAM = bytearray([0x02, 0x00])
STOP_ECG_STREAM = bytearray([0x03, 0x00])
START_PPI_STREAM = bytearray([0x02, 0x03])
STOP_PPI_STREAM = bytearray([0x03, 0x03])

## Polar H10 sampling frequencies ##
ACC_SAMPLING_FREQ = 200
ECG_SAMPLING_FREQ = 130

## Support functions

In [ ]:
ecg_data = []
ecg_session_time = []
acc_x_data = []
acc_y_data = []
acc_z_data = []
acc_session_time = []
t = []

## Bit conversion of the Hexadecimal stream
def data_conv(sender, data):
    
    ## ECG data packet
    if data[0] == 0x00:
        timestamp = convert_to_unsigned_long(data, 1, 8)/1e9
        step = 3
        samples = data[10:]
        offset = 0
        while offset < len(samples):
            ecg = convert_array_to_signed_int(samples, offset, step)
            offset += step
            ecg_data.extend([ecg])
            ecg_session_time.extend([timestamp])
            timestamp += 1/ECG_SAMPLING_FREQ
    
    ## Acceleration data packet
    elif data[0] == 0x02:
        timestamp = convert_to_unsigned_long(data, 1, 8)/1e9
        frame_type = data[9]
        resolution = (frame_type + 1) * 8
        step = math.ceil(resolution / 8.0)
        samples = data[10:]
        offset = 0
        while offset < len(samples):
            x = convert_array_to_signed_int(samples, offset, step)
            offset += step
            y = convert_array_to_signed_int(samples, offset, step)
            offset += step
            z = convert_array_to_signed_int(samples, offset, step)
            offset += step

            acc_x_data.extend([x])
            acc_y_data.extend([y])
            acc_z_data.extend([z])
            acc_session_time.extend([timestamp])
            timestamp += 1/ACC_SAMPLING_FREQ

## Conversion functions from bytes to numeric arrays
def convert_array_to_signed_int(data, offset, length):
    return int.from_bytes(
        bytearray(data[offset : offset + length]), byteorder="little", signed=True,
    )

def convert_to_unsigned_long(data, offset, length):
    return int.from_bytes(
        bytearray(data[offset : offset + length]), byteorder="little", signed=False,
    )

## Data recorder

In [ ]:
## Main function
async def main(address):
   
    try:  
        print("[OPEN] application opened...")
        
        client = BleakClient(address)
        await client.connect()
        print('BLE connection opened.')
        
        model_number = await client.read_gatt_char(MODEL_NBR_UUID)
        print(f'Model Number: {"".join(map(chr, model_number))}')
        
        manufacturer_name = await client.read_gatt_char(MANUFACTURER_NAME_UUID)
        print(f'Manufacturer: {"".join(map(chr, manufacturer_name))}')

        battery_level = await client.read_gatt_char(BATTERY_LEVEL_UUID)
        print(f'Battery Level: {int(battery_level[0])}%')

        att_read = await client.read_gatt_char(PMD_CONTROL)
        if att_read[0] == 0x0F:
            features = att_read[1]
            ECG_SUPPORTED = features&0b000001 != 0
            PPG_SUPPORTED = features&0b000010 != 0
            ACC_SUPPORTED = features&0b000100 != 0
            PPI_SUPPORTED = features&0b001000 != 0
            print('Supported measurement types:') 
            print('    ECG = ', ECG_SUPPORTED)
            print('    PPG = ', PPG_SUPPORTED)
            print('    ACC = ', ACC_SUPPORTED)
            print('    PPI = ', PPI_SUPPORTED) 
            
        ## Start streaming ECG data ##
        await client.write_gatt_char(PMD_CONTROL, ECG_WRITE)

        ## Start streaming accelerometer data ##
        await client.write_gatt_char(PMD_CONTROL, ACC_WRITE)

        ## Whenever there is data available, call data_conv
        await client.start_notify(PMD_DATA, data_conv)

        ## TypicalLY it takes 30 seconds to get the streaming to start
        print(f"Waiting for {WAIT_N_SECONDS} sec...")
        await asyncio.sleep(1)
        time.sleep(WAIT_N_SECONDS)
    
        print("Collecting data...")

        ## Create a graph
        fig = plt.figure(figsize=(15, 8))
        ax1 = fig.add_subplot(2, 1, 1)
        fig.suptitle("Live Data Stream on Polar-H10", fontsize=15)
        ax2 = fig.add_subplot(2, 1, 2)
        
        n1 = ECG_SAMPLING_FREQ
        n2 = ACC_SAMPLING_FREQ
        
        ## Data stream loop
        while n1 < REC_LENGTH*ECG_SAMPLING_FREQ:

            ## Collecting data for a second
            await asyncio.sleep(1)

            fig.suptitle(f'Live Data Stream on Polar-H10 {n1/ECG_SAMPLING_FREQ:6.0f} sec', fontsize=15)
            
            ## Update graphics for ECG
            ax1.clear()
            if ecg_session_time:
                t = np.array(ecg_session_time) - ecg_session_time[0]
                ax1.plot(t, ecg_data, label = 'ecg')
                ax1.set_xlim(t[-1]-6, t[-1])
                ax1.set_ylim(-2000, +2000)
                ax1.set_ylabel('Amplitude (uV)')
                ax1.legend()
            
            ## Update graphics for Acceleration
            ax2.clear()
            if acc_session_time:
                t = np.array(acc_session_time) - acc_session_time[0]
                ax2.plot(t, acc_x_data, label = 'x')
                ax2.plot(t, acc_y_data, label = 'y')
                ax2.plot(t, acc_z_data, label = 'z')
                ax2.set_xlim(t[-1]-6, t[-1])
                ax2.set_ylabel('Amplitude (mg)')
                ax2.set_xlabel('Time (sec)')
                ax2.legend()
            
            display(fig)
            clear_output(wait=True)

            n1 = n1 + ECG_SAMPLING_FREQ
            n2 = n2 + ACC_SAMPLING_FREQ
               
        ## Stop the stream once data is collected ##
        await client.stop_notify(PMD_DATA)
        print("Stopping data streaming...")
        
        ## Write ECG data to ECG_DATAFILE ##
        t = np.arange(len(ecg_data))/ECG_SAMPLING_FREQ
        df = pd.DataFrame({'t': t, 'ecg': ecg_data}, index = None)
        df.to_csv(ECG_DATAFILE, index = False, float_format = "%.6f")
        print(f'ECG data written to file: {ECG_DATAFILE}.')
   
        ## Write ACC data to ACC_DATAFILE ##
        t = np.arange(len(acc_x_data))/ACC_SAMPLING_FREQ
        df = pd.DataFrame({'t': t, 'x': acc_x_data, 'y': acc_y_data, 'z': acc_z_data}, index = None)
        df.to_csv(ACC_DATAFILE, index = False, float_format = "%.4f")
        print(f'ACC data written to file: {ACC_DATAFILE}.')
   
        
    except Exception as e:
        print(e)
        print(f"Exception: {e=}, {type(e)=}")
        raise
        
    finally:
        await client.disconnect()
        print("[CLOSED] application closed.")

## Run the main function asynchronously
asyncio.run(main(ADDRESS))

## Data check

- Read the datafiles and plot them

In [ ]:
## Read and plot ECG data
df_ecg = pd.read_csv(ECG_DATAFILE, index_col = 't')
df_ecg.plot(figsize = (15, 6));
plt.title(f'ECG file = {ECG_DATAFILE}')
plt.ylabel('Amplitude (uV)')
plt.xlabel('Time (s)')

## Read and plot Acceleration data
df_acc = pd.read_csv(ACC_DATAFILE, index_col = 't')
df_acc.plot(figsize = (15, 6));
plt.title(f'ACC file = {ACC_DATAFILE}')
plt.ylabel('Amplitude (mg)')
plt.xlabel('Time (s)')

plt.show()